# Setup of SageMaker Endpoint for DescriptiveWorld computer vision model

Authored by Blair Jones, 2021.11.01

Model built using yolov5

Code inspired by AWS-provided example at: https://aws.amazon.com/blogs/machine-learning/speed-up-yolov4-inference-to-twice-as-fast-on-amazon-sagemaker/



### Instructions

Before running the notebook, install yolov5 in the Sagemaker root folder using the yolov5 instructions at:  https://github.com/ultralytics/yolov5

and run "aws configure" at command line to setup aws CLI.

### Setup

In [1]:
import numpy as np
import time
import json
import requests
import boto3
import os
import sagemaker
from tqdm.notebook import tqdm

In [3]:
from sagemaker import get_execution_role
from sagemaker.session import Session

role = get_execution_role()
sess = Session()
region = sess.boto_region_name
bucket = 's3://descriptiveworld-models/CV_Models/'

In [4]:
print(region)
print(bucket)

us-west-2
s3://descriptiveworld-models/CV_Models/


In [5]:
# ensure that environment chosen for pytorch version >= 1.7 (as of 2021.11.01)

In [6]:
import torch
print(torch.__version__)

1.7.1


In [7]:
import sys
print(sys.version)

3.6.13 | packaged by conda-forge | (default, Feb 19 2021, 05:36:01) 
[GCC 9.3.0]


### Convert model to Torchscript

In [ ]:
!aws s3 cp s3://descriptiveworld-models/CV_Models/df2_11_large_20211021/weights/best.pt ../../yolov5/

In [8]:
%cd ../../yolov5
# not necessary to convert to torchscript
#!python ./bcj_export.py --weights ./best.pt

/home/ec2-user/SageMaker/yolov5


In [ ]:
!cp best.pt model.pt

In [ ]:
#!mkdir ./model
#!cp ./model.pt ./model/
#!mkdir ./model/code

In [ ]:
!mkdir ./code

In [ ]:
!cp ./utils/ -r ./code/utils/
!cp ./models/ -r ./code/models/
!cp ./data/ -r ./code/data/
!cp ./requirements.txt ./code/

In [9]:
!cp ./bcj_sm_ep_detect.py  ./code/

#### save model archive to s3

In [10]:
#!tar -czvf ./currentmodel.tar.gz ./current.torchscript.pt
!tar -czvf ./model.tar.gz ./model.pt
!aws s3 cp ./model.tar.gz s3://descriptiveworld-models/CV_Models/

./model.pt
upload: ./model.tar.gz to s3://descriptiveworld-models/CV_Models/model.tar.gz


### Create the model and endpoint

In [11]:
model_archive = './model.tar.gz'
prefix = 's3://descriptiveworld-models/CV_Models'
model_path = sess.upload_data(path=model_archive, key_prefix=prefix)
model_path

's3://sagemaker-us-west-2-769212126689/s3://descriptiveworld-models/CV_Models/model.tar.gz'

In [12]:
%%time
framework_version = '1.7.1'
py_version = 'py3'
instance_type = 'ml.t2.medium'
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

CPU times: user 8.33 ms, sys: 0 ns, total: 8.33 ms
Wall time: 14.5 ms


In [ ]:
%%time
# reference:  https://stackoverflow.com/questions/68150444/aws-sagemaker-fails-loading-pytorch-pth-weights
# reference:  https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/model.py
# reference:  https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/model.py

sm_model = PyTorchModel(model_data=model_path,
                       framework_version=framework_version,
                       role=role,
                       sagemaker_session=sess,
                       entry_point='bcj_sm_ep_detect.py',
                       source_dir='code',
                       py_version=py_version,
                       env={"COMPILEDMODEL": 'False', 'MMS_MAX_RESPONSE_SIZE': '100000000', 'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'})

dw_predictor = sm_model.deploy(initial_instance_count=1, 
                               instance_type=instance_type, 
                               endpoint_name='descriptiveworld-SageMaker-EndPoint')

print(sm_model.name)
print(dw_predictor.endpoint_name)

-----------!pytorch-inference-2021-11-15-14-11-50-590
pytorch-inference-2021-11-15-14-11-50-874
CPU times: user 7.12 s, sys: 755 ms, total: 7.88 s
Wall time: 5min 40s


### Test Endpoint

In [15]:
client = boto3.client('sagemaker-runtime', region_name=region)
sample_img_url = "https://media.gq.com/photos/60f9c697101cc04fad71e5cf/master/pass/BEST-BASICS-1.jpg"
content_type='JPEG'
body = requests.get(sample_img_url).content
response = client.invoke_endpoint(EndpointName=dw_predictor.endpoint_name, Body=body, ContentType=content_type)
pred_out = response['Body'].read().decode()

In [16]:
print(pred_out)

{"source-ref": "TBD", "num-detected-objects": 6, "bounding-box-attribute-name": {"image_size": [{"width": 2000, "height": 1125, "depth": 3}], "annotations": [{"class_id": 0, "left": 805, "top": 65, "width": 524, "height": 483}, {"class_id": 3, "left": 47, "top": 633, "width": 446, "height": 438}, {"class_id": 1, "left": 657, "top": 481, "width": 286, "height": 608}, {"class_id": 2, "left": 1094, "top": 579, "width": 465, "height": 505}, {"class_id": 1, "left": 1671, "top": 518, "width": 305, "height": 552}, {"class_id": 2, "left": 112, "top": 45, "width": 544, "height": 577}]}, "bounding-box-attribute-name-metadata": {"objects": [{"confidence": 0.25}, {"confidence": 0.28}, {"confidence": 0.31}, {"confidence": 0.42}, {"confidence": 0.48}, {"confidence": 0.49}], "class-map": {"0": "short sleeve top", "3": "shorts", "1": "trousers", "2": "long sleeve top"}, "type": "descriptiveworld/object-detection", "human-annotated": "no", "creation-date": "2021-11-15 14:18:49.785599", "job-name": "des

### Run Inference

In [ ]:
%%time
iters = 1000
warmup = 100
client = boto3.client('sagemaker-runtime', region_name=region)

content_type = 'application/x-image'

sample_img_url = "https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg"
body = requests.get(sample_img_url).content

dw_perf = []
  
for i in tqdm(range(iters)):
    t0 = time.time()
    response = client.invoke_endpoint(EndpointName=dw_predictor.endpoint_name, Body=body, ContentType=content_type)
    t1 = time.time()
    #convert to millis
    dw_elapsed = (t1-t0)*1000
    
    if warmup == 0:
        dw_perf.append(uncompiled_elapsed)
    else:
        print(f'warmup ({i}, {iters}) : dw - {dw_elapsed} ms')
        warmup = warmup - 1